In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from scipy import optimize as opt

In [2]:
matches = pd.read_csv("upcoming.csv", index_col=0)

## Cleaning and getting data ready to use.

In [3]:
#Encoding label we make result being a numeric value, which we can predict later. Instead of W, D, L.
yle = LabelEncoder()
matches['result'] = yle.fit_transform(matches['result'])

In [4]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,g-xg,npxg/sh,season,team
1,2022-08-14,22:00,La Liga,Matchweek 1,Sun,Away,2,2.0,1.0,Almería,...,Juan Martínez,Match Report,NaN,29.0,15.0,17.9,-0.4,0.09,2023,Real Madrid
2,2022-08-20,22:00,La Liga,Matchweek 2,Sat,Away,2,4.0,1.0,Celta Vigo,...,Jesús Gil,Match Report,NaN,15.0,6.0,18.7,0.6,0.13,2023,Real Madrid
3,2022-08-28,22:00,La Liga,Matchweek 3,Sun,Away,2,3.0,1.0,Espanyol,...,Mario Melero,Match Report,NaN,19.0,6.0,15.6,1.4,0.08,2023,Real Madrid
4,2022-09-03,16:15,La Liga,Matchweek 4,Sat,Home,2,2.0,1.0,Betis,...,José Sánchez,Match Report,NaN,22.0,5.0,17.2,-0.6,0.12,2023,Real Madrid
6,2022-09-11,14:00,La Liga,Matchweek 5,Sun,Home,2,4.0,1.0,Mallorca,...,Jorge Figueroa,Match Report,NaN,22.0,7.0,21.5,2.4,0.07,2023,Real Madrid


In [5]:
#Using pandas sweets.
matches["date"] = pd.to_datetime(matches["date"])

#Creating numeric values for Home and Away, so that machine can learn from it.
matches['venue_c'] = matches['venue'].astype('category').cat.codes

#Creating unique code for each opponent squad.
matches['opp_c'] = matches['opponent'].astype('category').cat.codes

#Creating a normalized hour time.
matches['hour'] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

#Monday = 0, Tuesday = 1, etc.
matches['day_c'] = matches['date'].dt.dayofweek

matches = matches.drop(columns=['round', 'day', 'comp'])

In [6]:
matches = matches[['date', 'result', 'venue', 'gf', 'ga', 'xg', 'xga',
       'poss', 'sh', 'sot', 'dist', 'g-xg', 'team', 'opponent', 'opp_c']]

#Making all teams names correct.
matches = matches.replace("Atlético Madrid", "Atletico Madrid")
matches = matches.replace("Alavés", "Alaves")
matches = matches.replace("Leganés", "Leganes")
matches = matches.replace("Málaga", "Malaga")
matches = matches.replace("La Coruña", "Deportivo La Coruna")
matches = matches.replace("Atlético Madrid", "Atletico Madrid")
matches = matches.replace("Real Betis", "Betis")


In [7]:
matches.head()

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c
1,2022-08-14,2,Away,2.0,1.0,2.4,0.6,67.0,29.0,15.0,17.9,-0.4,Real Madrid,Almería,1
2,2022-08-20,2,Away,4.0,1.0,3.4,1.3,50.0,15.0,6.0,18.7,0.6,Real Madrid,Celta Vigo,6
3,2022-08-28,2,Away,3.0,1.0,1.6,1.1,65.0,19.0,6.0,15.6,1.4,Real Madrid,Espanyol,10
4,2022-09-03,2,Home,2.0,1.0,2.6,0.5,49.0,22.0,5.0,17.2,-0.6,Real Madrid,Betis,5
6,2022-09-11,2,Home,4.0,1.0,1.6,0.4,68.0,22.0,7.0,21.5,2.4,Real Madrid,Mallorca,19


In [8]:
at_home = matches['venue'] == "Home"
matches_home = matches[at_home].sort_values(['team','date'])
matches_home['xg_ha'] = matches_home['xg'].rolling(38, closed='left').mean()
matches_home['xga_ha'] = matches_home['xga'].rolling(38, closed='left').mean()
matches_home['g-xg_ha'] = matches_home['g-xg'].rolling(38, closed='left').mean()

In [9]:
at_away = matches['venue'] == "Away"
matches_away = matches[at_away].sort_values(['team', 'date'])
matches_away['xg_ha'] = matches_away['xg'].rolling(38, closed='left').mean()
matches_away['xga_ha'] = matches_away['xga'].rolling(38, closed='left').mean()
matches_away['g-xg_ha'] = matches_away['g-xg'].rolling(38, closed='left').mean()

In [10]:
matches_home[4:14]

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c,xg_ha,xga_ha,g-xg_ha
10,2017-10-28,1,Home,1.0,2.0,0.8,1.1,45.0,18.0,7.0,21.1,0.2,Alaves,Valencia,26,NaN,NaN,NaN
11,2017-11-04,2,Home,1.0,0.0,1.2,0.8,35.0,17.0,5.0,22.8,-0.2,Alaves,Espanyol,10,NaN,NaN,NaN
13,2017-11-25,1,Home,1.0,2.0,0.4,0.7,44.0,6.0,2.0,20.0,0.6,Alaves,Eibar,8,NaN,NaN,NaN
16,2017-12-08,2,Home,2.0,0.0,3.4,0.3,36.0,23.0,9.0,13.7,-1.4,Alaves,Las Palmas,16,NaN,NaN,NaN
18,2017-12-21,2,Home,1.0,0.0,1.7,0.3,41.0,13.0,5.0,17.9,-0.7,Alaves,Malaga,20,NaN,NaN,NaN
22,2018-01-14,2,Home,1.0,0.0,0.4,0.6,39.0,11.0,4.0,20.2,0.6,Alaves,Sevilla,25,NaN,NaN,NaN
24,2018-01-21,0,Home,2.0,2.0,2.5,1.4,48.0,14.0,6.0,17.1,-0.5,Alaves,Leganes,17,NaN,NaN,NaN
27,2018-02-03,2,Home,2.0,1.0,0.7,1.5,28.0,9.0,3.0,13.2,1.3,Alaves,Celta Vigo,6,NaN,NaN,NaN
29,2018-02-17,2,Home,1.0,0.0,1.1,0.6,48.0,15.0,3.0,16.6,-0.1,Alaves,Deportivo La Coruna,15,NaN,NaN,NaN
31,2018-03-01,2,Home,1.0,0.0,2.5,0.4,64.0,14.0,5.0,16.6,-1.5,Alaves,Levante,18,NaN,NaN,NaN


In [11]:
matches_away[4:14]

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c,xg_ha,xga_ha,g-xg_ha
8,2017-10-21,1,Away,0.0,2.0,0.7,0.8,33.0,7.0,3.0,18.8,-0.7,Alaves,Betis,5,NaN,NaN,NaN
12,2017-11-18,1,Away,1.0,4.0,0.7,2.8,57.0,10.0,6.0,16.7,0.3,Alaves,Getafe,11,NaN,NaN,NaN
15,2017-12-04,2,Away,3.0,2.0,1.7,0.6,45.0,8.0,3.0,25.5,1.3,Alaves,Girona,12,NaN,NaN,NaN
17,2017-12-16,1,Away,0.0,1.0,0.5,0.9,37.0,11.0,0.0,19.3,-0.5,Alaves,Atletico Madrid,3,NaN,NaN,NaN
20,2018-01-07,1,Away,0.0,2.0,0.2,1.8,39.0,8.0,1.0,27.9,-0.2,Alaves,Athletic Club,2,NaN,NaN,NaN
26,2018-01-28,1,Away,1.0,2.0,0.6,1.8,23.0,7.0,4.0,18.3,0.4,Alaves,Barcelona,4,NaN,NaN,NaN
28,2018-02-10,2,Away,2.0,1.0,0.9,2.0,31.0,7.0,4.0,12.5,1.1,Alaves,Villarreal,28,NaN,NaN,NaN
30,2018-02-24,1,Away,0.0,4.0,0.9,2.9,37.0,12.0,5.0,17.8,-0.9,Alaves,Real Madrid,23,NaN,NaN,NaN
32,2018-03-04,1,Away,1.0,2.0,0.3,2.0,40.0,4.0,1.0,12.2,0.7,Alaves,Real Sociedad,24,NaN,NaN,NaN
34,2018-03-17,1,Away,1.0,3.0,1.2,2.7,39.0,17.0,5.0,21.0,-0.2,Alaves,Valencia,26,NaN,NaN,NaN


In [12]:
c = [matches_home, matches_away]

In [13]:
grouping = pd.concat(c)

In [14]:
grouping.sort_values('date').round(2)

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c,xg_ha,xga_ha,g-xg_ha
0,2017-08-18,2,Home,1.0,0.0,1.4,1.2,54.0,14.0,3.0,20.8,-0.4,Leganes,Alaves,0,1.06,1.28,-0.27
0,2017-08-18,1,Away,0.0,1.0,0.3,1.9,52.0,5.0,4.0,22.9,-0.3,Las Palmas,Valencia,26,0.95,1.53,-0.00
0,2017-08-18,2,Home,1.0,0.0,1.9,0.3,48.0,22.0,6.0,15.9,-0.9,Valencia,Las Palmas,16,0.96,1.29,0.47
0,2017-08-18,1,Away,0.0,1.0,1.2,1.4,46.0,8.0,3.0,21.7,-1.2,Alaves,Leganes,17,NaN,NaN,NaN
0,2017-08-19,0,Home,2.0,2.0,2.1,0.8,53.0,14.0,6.0,16.8,-0.1,Girona,Atletico Madrid,3,1.05,0.85,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,2023-06-04,1,Away,1.0,2.0,1.2,1.6,56.0,9.0,3.0,17.8,-0.2,Girona,Osasuna,21,1.04,1.54,-0.02
37,2023-06-04,1,Away,2.0,3.0,1.0,2.6,61.0,12.0,3.0,17.6,1.0,Rayo Vallecano,Mallorca,19,0.99,1.93,0.28
43,2023-06-04,0,Away,1.0,1.0,0.5,1.6,54.0,12.0,1.0,20.6,0.5,Sevilla,Real Sociedad,24,0.84,1.37,0.08
37,2023-06-04,1,Home,0.0,2.0,0.9,1.4,54.0,12.0,5.0,13.9,-0.9,Osasuna,Girona,12,1.09,1.20,-0.49


In [15]:
matches = grouping.round(2)

In [16]:
matches.head()

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c,xg_ha,xga_ha,g-xg_ha
1,2017-08-26,1,Home,0.0,2.0,0.8,2.4,27.0,11.0,2.0,20.9,-0.8,Alaves,Barcelona,4,NaN,NaN,NaN
3,2017-09-17,1,Home,0.0,3.0,0.4,1.9,48.0,16.0,2.0,24.1,-0.4,Alaves,Villarreal,28,NaN,NaN,NaN
5,2017-09-23,1,Home,1.0,2.0,0.7,3.1,33.0,8.0,3.0,18.5,0.3,Alaves,Real Madrid,23,NaN,NaN,NaN
7,2017-10-14,1,Home,0.0,2.0,0.5,0.7,27.0,8.0,0.0,19.7,-0.5,Alaves,Real Sociedad,24,NaN,NaN,NaN
10,2017-10-28,1,Home,1.0,2.0,0.8,1.1,45.0,18.0,7.0,21.1,0.2,Alaves,Valencia,26,NaN,NaN,NaN


In [17]:
is_home = matches['venue'] == "Home"
home = matches[is_home]
away = matches[~is_home]

In [18]:
#Setting up our data, so we can add opponents value at the current time to our DataFrame.
home = home.merge(
    away[["date", "opponent", "team", "xg", "xga", "sh", "poss", "g-xg", "xg_ha", "xga_ha", "g-xg_ha"]],
    left_on=["date", "team", "opponent"],
    right_on=["date", "opponent", "team"],
    how="outer",
    suffixes=("", "_opp"),
).drop(columns=["opponent_opp", "team_opp"])

In [19]:
home = home.dropna()

In [20]:
matches = home

In [21]:
matches.sort_values('date')[500:550]

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,...,xga_ha,g-xg_ha,xg_opp,xga_opp,sh_opp,poss_opp,g-xg_opp,xg_ha_opp,xga_ha_opp,g-xg_ha_opp
788,2019-01-06,0.0,Home,0.0,0.0,1.8,0.9,54.0,24.0,7.0,...,1.08,-0.06,0.9,1.8,7.0,46.0,-0.9,1.23,1.32,-0.39
1889,2019-01-06,0.0,Home,1.0,1.0,1.5,0.8,57.0,14.0,4.0,...,1.09,-0.28,0.8,1.5,11.0,43.0,0.2,1.57,1.05,0.20
1015,2019-01-06,1.0,Home,1.0,2.0,1.4,2.0,24.0,10.0,2.0,...,1.20,-0.14,2.0,1.4,13.0,76.0,0.0,1.52,1.13,0.40
655,2019-01-07,1.0,Home,1.0,2.0,1.1,1.6,65.0,16.0,3.0,...,1.21,0.27,1.6,1.1,8.0,35.0,0.4,0.85,1.23,-0.14
1586,2019-01-11,2.0,Home,4.0,2.0,2.0,1.2,39.0,11.0,8.0,...,1.22,-0.45,1.2,2.0,11.0,61.0,0.8,0.88,1.59,0.39
1300,2019-01-12,2.0,Home,1.0,0.0,1.9,0.5,49.0,14.0,7.0,...,1.16,-0.27,0.5,1.9,8.0,51.0,-0.5,1.00,1.64,0.18
2194,2019-01-12,1.0,Home,1.0,2.0,1.9,0.9,57.0,16.0,6.0,...,0.93,-0.34,0.9,1.9,9.0,43.0,1.1,0.95,1.31,-0.03
2004,2019-01-12,0.0,Home,1.0,1.0,2.3,0.1,63.0,16.0,4.0,...,1.17,0.03,0.1,2.3,5.0,37.0,0.9,1.26,1.06,0.53
256,2019-01-13,2.0,Home,1.0,0.0,2.3,0.7,58.0,18.0,4.0,...,0.91,0.37,0.7,2.3,7.0,42.0,-0.7,0.98,1.59,0.07
370,2019-01-13,2.0,Home,3.0,0.0,1.5,0.8,58.0,7.0,6.0,...,0.95,0.31,0.8,1.5,15.0,42.0,-0.8,0.96,1.39,-0.17


In [22]:
grouped_matches = matches.groupby('team')

In [23]:
group = grouped_matches.get_group("Real Madrid")

In [24]:
#Short function to get rolling averages as our predictors from last 3 matches.
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(35, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [25]:
#cols = ['poss', 'sh', 'xg', 'xga', 'g-xg', 'poss_opp', 'sh_opp', 'xg_opp', 'xga_opp', 'g-xg_opp']
predictors = ['xg_ha', 'xga_ha', 'xg_ha_opp', 'xga_ha_opp', 'g-xg_ha', 'g-xg_ha_opp']
cols = ['xg', 'xga', 'g-xg', 'xg_opp', 'xga_opp', 'g-xg_opp']
new_cols = [f"{c}R" for c in cols]

In [26]:
#Getting the rolling averages.
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

In [27]:
matches_rolling = matches_rolling[matches_rolling['date'] <= '2022-11-14']

In [28]:
df = matches_rolling
df = df.sort_values('date')

In [29]:
df[['date', 'result', 'g-xg_ha', 'xg_ha', 'team', 'opponent', 'xg_ha_opp', 'g-xg_ha_opp']].round(2)[-20:]

,date,result,g-xg_ha,xg_ha,team,opponent,xg_ha_opp,g-xg_ha_opp
1039,2022-10-30,2.0,0.31,1.30,Valencia,Barcelona,1.96,0.22
965,2022-10-30,0.0,0.29,1.32,Sevilla,Rayo Vallecano,0.90,-0.01
728,2022-10-30,1.0,-0.21,1.18,Osasuna,Valladolid,0.83,-0.17
819,2022-10-30,2.0,0.36,1.77,Real Madrid,Girona,0.98,0.10
154,2022-11-06,1.0,0.24,1.47,Atletico Madrid,Espanyol,0.85,-0.06
893,2022-11-06,2.0,-0.30,1.62,Real Sociedad,Valencia,1.00,0.08
300,2022-11-06,2.0,0.18,1.50,Betis,Sevilla,1.12,-0.02
373,2022-11-06,1.0,0.20,1.17,Celta Vigo,Osasuna,0.93,-0.01
1146,2022-11-06,0.0,-0.04,1.77,Villarreal,Mallorca,0.83,0.01
745,2022-11-06,1.0,-0.01,1.33,Rayo Vallecano,Real Madrid,1.76,0.19


## Now let's set up our model, train, test and see the results.

In [30]:
#Setting up our neural network model with 12 input layers, a lot of hidden layers and 3 output layers (Win, Draw, Lose). 

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(12,)),
    keras.layers.Dense(36, activation="relu"),
    keras.layers.Dense(105, activation="relu"),
    keras.layers.Dense(315, activation="relu"),
    keras.layers.Dense(150, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
#Setting up training and testing slices.
train = df[df['date'] <= '2022-09-25']
test = df[df['date'] > '2022-09-25']

In [33]:
model.fit(train[predictors + new_cols], train['result'], epochs=50, verbose=0)

In [34]:
test_loss, test_acc = model.evaluate(test[predictors + new_cols], test['result'])

2/2 [==============================] - 0s 4ms/step - loss: 1.1140 - accuracy: 0.3750


In [35]:
prediction = model.predict(test[predictors + new_cols])

2/2 [==============================] - 0s 2ms/step


In [36]:
prediction[:5]     #Draw, Away, Home   <- predictions are in that order.

array([[0.41085073, 0.16820991, 0.4209394 ],
       [0.41839144, 0.20023046, 0.38137802],
       [0.34230345, 0.5086668 , 0.14902964],
       [0.4071941 , 0.33887583, 0.2539301 ],
       [0.22279614, 0.6229809 , 0.1542229 ]], dtype=float32)

In [37]:
#Adding the probabilities to our sweet DataFrame.
test['home'] = prediction[:,2]
test['draw'] = prediction[:,0]
test['away'] = prediction[:,1]

C:\Users\PC\AppData\Local\Temp\ipykernel_22292\127324621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['home'] = prediction[:,2]
C:\Users\PC\AppData\Local\Temp\ipykernel_22292\127324621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['draw'] = prediction[:,0]
C:\Users\PC\AppData\Local\Temp\ipykernel_22292\127324621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [38]:
test = test[['date', 'team', 'opponent', 'home', 'draw', 'away']].round(2)

In [39]:
#Now we have the probabilities that our model is set for each even to happen in a certain match.
test.sort_values('date')[:5]

,date,team,opponent,home,draw,away
555,2022-10-01,Getafe,Valladolid,0.42,0.41,0.17
962,2022-10-01,Sevilla,Atletico Madrid,0.38,0.42,0.20
688,2022-10-01,Mallorca,Barcelona,0.15,0.34,0.51
370,2022-10-02,Celta Vigo,Betis,0.25,0.41,0.34
573,2022-10-02,Girona,Real Sociedad,0.15,0.22,0.62


In [40]:
test['home'] = 1/test['home']
test['draw'] = 1/test['draw']
test['away'] = 1/test['away']

In [41]:
test = test.sort_values('date').round(2)

In [42]:
test[:15]

,date,team,opponent,home,draw,away
555,2022-10-01,Getafe,Valladolid,2.38,2.44,5.88
962,2022-10-01,Sevilla,Atletico Madrid,2.63,2.38,5.00
688,2022-10-01,Mallorca,Barcelona,6.67,2.94,1.96
370,2022-10-02,Celta Vigo,Betis,4.00,2.44,2.94
573,2022-10-02,Girona,Real Sociedad,6.67,4.55,1.61
482,2022-10-02,Espanyol,Valencia,2.27,3.85,3.33
816,2022-10-02,Real Madrid,Osasuna,1.32,5.26,20.00
743,2022-10-03,Rayo Vallecano,Elche,1.47,7.69,5.26
726,2022-10-07,Osasuna,Valencia,4.00,3.45,2.13
556,2022-10-08,Getafe,Real Madrid,5.88,1.54,5.88


In [43]:
#Almeria, Valladolid, Girona are new teams in LaLiga, so the stats are not useful with them.